In [126]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model,save_model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [94]:
df=pd.read_excel('LungCap.xls')
df.head()

,LungCap(cc),Age( years),Height(inches),Smoke,Gender,Caesarean
0,6.475,6,62.1,no,male,no
1,10.125,18,74.7,yes,female,no
2,9.550,16,69.7,no,female,yes
3,11.125,14,71.0,no,male,no
4,4.800,5,56.9,no,male,no


In [95]:
df.shape, df['Height(inches)'].unique().shape,df['LungCap(cc)'].unique().shape

((725, 6), (274,), (342,))

In [96]:
df.drop_duplicates(subset=['LungCap(cc)'],inplace=True)
df.info

<bound method DataFrame.info of      LungCap(cc)  Age( years)  Height(inches) Smoke  Gender Caesarean
0          6.475            6            62.1    no    male        no
1         10.125           18            74.7   yes  female        no
2          9.550           16            69.7    no  female       yes
3         11.125           14            71.0    no    male        no
4          4.800            5            56.9    no    male        no
..           ...          ...             ...   ...     ...       ...
703        3.600            7            53.9    no    male        no
707        5.275           12            58.5    no  female       yes
716        7.075           11            66.7    no    male       yes
718        7.175           17            68.8    no    male       yes
722        3.850           11            60.5   yes  female        no

[342 rows x 6 columns]>

In [97]:
# df['smoke_labelled']=0
# df['Caesarean_labelled']=0
# df['Gender_labelled']=0

df['smoke_labelled'] = df['Smoke'].apply(lambda x: 1 if x == 'yes' else 0)
df['Caesarean_labelled'] = df['Caesarean'].apply(lambda x: 1 if x == 'yes' else 0)
df['Gender_labelled']=df['Gender'].apply(lambda x:1 if x=='female' else 0)
df.drop(columns=['Smoke','Caesarean','Gender'],inplace=True)
df.head()

,LungCap(cc),Age( years),Height(inches),smoke_labelled,Caesarean_labelled,Gender_labelled
0,6.475,6,62.1,0,0,0
1,10.125,18,74.7,1,0,1
2,9.550,16,69.7,0,1,1
3,11.125,14,71.0,0,0,0
4,4.800,5,56.9,0,0,0


In [98]:
df['Tidal Volume']=df['LungCap(cc)']/10.0
df.head()

,LungCap(cc),Age( years),Height(inches),smoke_labelled,Caesarean_labelled,Gender_labelled,Tidal Volume
0,6.475,6,62.1,0,0,0,0.6475
1,10.125,18,74.7,1,0,1,1.0125
2,9.550,16,69.7,0,1,1,0.9550
3,11.125,14,71.0,0,0,0,1.1125
4,4.800,5,56.9,0,0,0,0.4800


In [99]:
data=df.drop(columns=['LungCap(cc)'])
data.head()

,Age( years),Height(inches),smoke_labelled,Caesarean_labelled,Gender_labelled,Tidal Volume
0,6,62.1,0,0,0,0.6475
1,18,74.7,1,0,1,1.0125
2,16,69.7,0,1,1,0.9550
3,14,71.0,0,0,0,1.1125
4,5,56.9,0,0,0,0.4800


In [119]:
# data_mean = data.mean(axis=0)
# data_std = data.std(axis=0)
# data = (data - data_mean) / data_std

In [109]:
latent_dim = 100
data_dim = df.shape[1]

In [113]:
generator_input = Input(shape=(latent_dim,))
x = Dense(128, activation='relu')(generator_input)
x = Dense(256, activation='relu')(x)
x = Dense(data_dim, activation='tanh')(x)
generator = Model(generator_input, x)

# Discriminator
discriminator_input = Input(shape=(data_dim,)) 
x = Dense(256, activation='relu')(discriminator_input)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)
discriminator = Model(discriminator_input, x)

# Combined model (GAN)
gan_input = Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = Model(gan_input, gan_output)

# Compile discriminator
discriminator.compile(optimizer=Adam(learning_rate=0.0002), loss='binary_crossentropy', metrics=['accuracy'])

# Compile GAN
discriminator.trainable = False
gan.compile(optimizer=Adam(learning_rate=0.0002), loss='binary_crossentropy')

In [114]:
batch_size = 32
epochs = 10000
sample_interval = 1000

# Training loop
for epoch in range(epochs):
    # Train discriminator
    idx = np.random.randint(0, data.shape[0], batch_size)
    real_data = data.iloc[idx]
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    gen_data = generator.predict(noise)
    d_loss_real = discriminator.train_on_batch(real_data, np.ones((batch_size, 1)))
    d_loss_fake = discriminator.train_on_batch(gen_data, np.zeros((batch_size, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    
    # Train generator
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))
    
    # Print progress
    if epoch % 100 == 0:
        print(f"Epoch: {epoch}, D loss: {d_loss[0]}, G loss: {g_loss}")

    # Save generated samples
    if epoch % sample_interval == 0:
        noise = np.random.normal(0, 1, (5, latent_dim))
        gen_samples = generator.predict(noise)
        gen_samples = gen_samples * data_std + data_mean  # Denormalize
        gen_samples_df = pd.DataFrame(gen_samples, columns=data.columns)
        print(gen_samples_df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


ValueError: Input 0 of layer "functional_14" is incompatible with the layer: expected shape=(None, 7), found shape=(32, 6)

In [118]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define your generator architecture
latent_dim = 100
generator_input = Input(shape=(latent_dim,))
x = Dense(128, activation='relu')(generator_input)
generator_output = Dense(data_dim)(x)  # data_dim is the number of features in your dataset
generator = Model(generator_input, generator_output)

# Define your discriminator architecture
discriminator_input = Input(shape=(data_dim,))
x = Dense(128, activation='relu')(discriminator_input)
discriminator_output = Dense(1, activation='sigmoid')(x)
discriminator = Model(discriminator_input, discriminator_output)

# Compile the discriminator
discriminator.compile(optimizer=Adam(learning_rate=0.0002), loss='binary_crossentropy', metrics=['accuracy'])

# Freeze the discriminator during generator training
discriminator.trainable = False

# Define the GAN model
gan_input = Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = Model(gan_input, gan_output)

# Compile the GAN
gan.compile(optimizer=Adam(learning_rate=0.0002), loss='binary_crossentropy')

# Load and preprocess your dataset
# ...
batch_size = 32
epochs = 10000
sample_interval = 1000

# Training loop
for epoch in range(epochs):
    # Train discriminator
    idx = np.random.randint(0, data.shape[0], batch_size)
    real_data = data.iloc[idx]
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    gen_data = generator.predict(noise)
    d_loss_real = discriminator.train_on_batch(real_data, np.ones((batch_size, 1)))
    d_loss_fake = discriminator.train_on_batch(gen_data, np.zeros((batch_size, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    
    # Train generator
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))
    
    # Print progress
    if epoch % 100 == 0:
        print(f"Epoch: {epoch}, D loss: {d_loss[0]}, G loss: {g_loss}")

    # Save generated samples
    if epoch % sample_interval == 0:
        noise = np.random.normal(0, 1, (5, latent_dim))
        gen_samples = generator.predict(noise)
        gen_samples = gen_samples * data_std + data_mean  # Denormalize
        gen_samples_df = pd.DataFrame(gen_samples, columns=data.columns)
        print(gen_samples_df)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


ValueError: Input 0 of layer "functional_38" is incompatible with the layer: expected shape=(None, 7), found shape=(32, 6)

In [124]:
X_train, X_val, y_train, y_val = train_test_split(data.drop(columns=['Tidal Volume']),data['Tidal Volume'], test_size=0.2, random_state=42)
X_train.shape

(273, 5)

In [139]:
model = Sequential([
    Dense(128, activation='relu',input_shape=(5,)),
    Dropout(0.5),
    Dense(64,activation='relu'),
    Dense(32,activation='relu'),
    Dropout(0.5),
    Dense(1, activation='relu'),
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

C:\Users\hardi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [140]:
model.fit(X_train, y_train, batch_size=20, epochs=15, validation_data=(X_val, y_val))
model.save('Model.keras')

Epoch 1/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.0000e+00 - loss: 0.8599 - val_accuracy: 0.0000e+00 - val_loss: -4.9434
Epoch 2/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: -4.2453 - val_accuracy: 0.0000e+00 - val_loss: -5.5396
Epoch 3/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: -4.4320 - val_accuracy: 0.0000e+00 - val_loss: -5.7349
Epoch 4/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: -5.9520 - val_accuracy: 0.0000e+00 - val_loss: -6.2854
Epoch 5/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: -5.8835 - val_accuracy: 0.0000e+00 - val_loss: -6.3589
Epoch 6/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: -6.0948 - val_accuracy: 0.0000e+00 - val_loss: -6.2027
Epoch 7/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: -5.3050 - val_accuracy: 0.0000e+00 - val_loss: -6.3169
Epoch 8/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/ste